In [ ]:
#%pip install jira pandas
from jira import JIRA
import pandas as pd
from IPython.display import clear_output
#%pip install "ipython== 8.2.0"
#%pip install "ipython-genutils == 0.2.0"
#%pip install "ipywidgets  == 7.6.5"

Reading the csv files to get the lenght!

In [ ]:
df_query = pd.read_csv(r'C:\Users\data\Desktop\Google Shopping Automation\fetced_data.csv')
df_beforeS1 = pd.read_csv(r'C:\Users\data\Desktop\Google Shopping Automation\beforeS1.csv')
df_nosearch = pd.read_csv(r'C:\Users\data\Desktop\Google Shopping Automation\not_for_search.csv')
df_afterS1 = pd.read_csv(r'C:\Users\data\Desktop\Google Shopping Automation\google_shop_competitors.csv')
df_afterS2 = pd.read_csv(r'C:\Users\data\Desktop\Google Shopping Automation\GS_final.csv')

In [ ]:
country_code = country_code.upper()

In [ ]:
#https://tgndata.atlassian.net/rest/greenhopper/1.0/rapidview
#https://tgndata.atlassian.net/rest/greenhopper/latest/sprintquery/10?includeHistoricSprints=true&includeFutureSprints=true
#board id: 10
#last sprint_id = 47

Setting up the credentials

In [ ]:
jira_url = 'https://tgndata.atlassian.net/'
username = 'xxx'
api_token = 'xxx'

In [ ]:
#getting the active sprint id
import requests
from requests.auth import HTTPBasicAuth

url = jira_url + 'rest/greenhopper/latest/sprintquery/10?includeHistoricSprints=true&includeFutureSprints=true'

r = requests.get(url, auth=HTTPBasicAuth(username, api_token))
data = r.json()

active_sprints = data['sprints']
active_sprint_id = None

for sprint in active_sprints:
    if sprint['state'] == 'ACTIVE':
        sprint_id = sprint['id']
        break

In [ ]:
#create Jira connection
jira = JIRA(server=jira_url, basic_auth=(username, api_token))

Task Details

In [ ]:
project_key = 'DS'
issue_type = 'Story'
summary = 'Daily Google Procedure ' + str(country_code) #This is a sample task created from Python using jira integration with REST API
#reporter = 'Giannis Karousos' #I have this one automated, no need
assignee = 'xxx'
description = 'From the Database we got a file with ' + str(len(df_query)) + ' rows. | After some cleaning {color:blue} we got ' + '{*}' + str(len(df_beforeS1)) + '{*}' + ' to search for Stage 1 {color} while there were ' + str(len(df_nosearch)) + ' rows ready for the next stage that we do not have to search. | {color:blue}The rows we got after the Stage 1 are: ' + '{*}' + str(len(df_afterS1)) + '{*}' + '{color}. | Summarizing the process, {color:blue} the final file after completing the Stage 2 is: ' + '{*}' + str(len(df_afterS2)) + '{*}' + '.{color}' + ' Bellow you will find the CSVs needed!'
#priority = 'High'
labels = ['General_Data_Renewal'] #Weekly_Data_Renew, Assortment, General_Data_Renewal
#sprint_id = 50 #I must renew this one everytime a new sprint arrives

In [ ]:
#create the issue
issue_dict = {
    'project': {'key': project_key},
    'issuetype': {'name': issue_type},
    'summary': summary,
    'description': description,
    'assignee': {'id': assignee},
    'labels': labels,
    'customfield_10020': sprint_id #the customfield_10020 is related to our board which has an number id 10 with the latest sprint id
}

Connection established

In [ ]:
issue = jira.create_issue(fields = issue_dict)

Uploading CSV file on the issue

In [ ]:
#list of CSV file paths
csv_files = [
    r'C:\Users\data\Desktop\Google Shopping Automation\fetced_data.csv',
    r'C:\Users\data\Desktop\Google Shopping Automation\google_shop_competitors.csv',
    r'C:\Users\data\Desktop\Google Shopping Automation\variance.csv',
    r'C:\Users\data\Desktop\Google Shopping Automation\GS_images.csv',
    r'C:\Users\data\Desktop\Google Shopping Automation\GS_final.csv'
]

#iterate over each CSV file
for csv_file in csv_files:
    #upload the CSV file as an attachment
    attachment = jira.add_attachment(issue=issue, attachment=csv_file)

print('The files are up in the task!')

Creating the group by from the fetched data

In [ ]:
fetched_path = r'C:\Users\data\Desktop\Google Shopping Automation\fetced_data.csv'
stage1_answers = r'C:\Users\data\Desktop\Google Shopping Automation\GS_final.csv'
df_fetched = pd.read_csv(fetched_path)
df_fetched.head()

In [ ]:
stage1_answers = pd.read_csv(stage1_answers)
stage1_answers = stage1_answers[['PID']].drop_duplicates(keep='first')

stage1_answers.head()

In [ ]:
df_fetched['found'] = df_fetched['PID'].apply(lambda x: 'yes' if x in stage1_answers['PID'].values else 'no')
grouped_df = df_fetched.groupby('name').agg(count=('found', 'size'), found_yes=('found', lambda x: (x == 'yes').sum())).reset_index()
grouped_df

In [ ]:
#%pip install matplotlib
#%pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties

# Sort the DataFrame by the 'count' column in ascending order
grouped_df = grouped_df.sort_values('count', ascending=True)

# Define a color palette with the number of columns in the DataFrame
color_palette = sns.color_palette("Set2", len(grouped_df.columns))

# Create a stacked bar plot
plt.barh(grouped_df['name'], grouped_df['found_yes'], color=color_palette[0])
plt.barh(grouped_df['name'], grouped_df['count'] - grouped_df['found_yes'], left=grouped_df['found_yes'], color=color_palette[1])

plt.xlabel("Request's Count", fontweight='bold')
plt.ylabel("Client", fontweight='bold')
plt.title("Client's Requests", fontweight='bold', fontsize=18)

# Add legend
plt.legend(['Found: Yes', 'Found: No'])

# Add vertical grid lines
plt.grid(axis='x', linestyle='--', alpha=0.3)

# Add count labels next to each bar
for i in range(len(grouped_df)):
    plt.text(grouped_df['count'].iloc[i], i, f"{grouped_df['found_yes'].iloc[i]}/{grouped_df['count'].iloc[i]}",
             ha='left', va='center', fontweight='bold')

# Adjust the plot layout to make room for the table
plt.subplots_adjust(left=0.3, bottom=0.3, right=0.9, top=0.9)  # Adjust the values to modify the margins

# Display the plot
plt.savefig('client requests&findings.png', format='png', dpi=300)

In [ ]:
plot_filename = r'C:\Users\data\Desktop\Google Shopping Automation\client requests&findings.png'  # Replace with the filename used in step 1

with open(plot_filename, 'rb') as f:
    jira.add_attachment(issue=issue, attachment=f)

print('The image client requests&finding is up!')

In [ ]:
print("The ticket number is: " + str(issue.key))

In [ ]:
clear_output()